In [ ]:
import pandas as pd
import pymysql
import getpass

In [ ]:
pd.set_option("max_colwidth", 100)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [ ]:
# Let's connect to our database
username = "" # Replace with your MIMIC username
conn = pymysql.connect(host="35.233.174.193",port=3306,
                       user=username,
                       passwd=getpass.getpass(
                           "Enter password for MIMIC2 database"
                       ),
                       db='mimic2')

# Diagnoses
The table `icd9` contains the conditions which patients are assigned diagnoses with. ICD-9 codes are a standardized terminology where each code represents a very specific disease. ICD-9 has been replaced with ICD-10 coding, but historical data like MIMIC still contains ICD-9. 

In [ ]:
query = """
SELECT * FROM icd9
LIMIT 10;
"""
df = pd.read_sql(query, conn)
df.head(10)

### TODO
Find all **unique** ICD-9 codes and descriptions which contain the word **"diabetes"**. Use the `LIKE %...%` syntax in SQL.

In [ ]:
query = """
SELECT ____ code, description
FROM ____
WHERE ____ ____ ____
ORDER BY code;
"""
diabetes = pd.read_sql(query, conn)

In [ ]:
diabetes

## ICD hierarchy
ICD codes are organizaed hierarchically. Codes started with the same characters are going to have similar meanings. 

### TODO
1. Add a column to the DataFrame `diabetes` which contains the first 3 letters of the ICD-9 code
2. Then get the count of the first three characters and their counts
3. Finally, subset the dataframe using boolean indexing to get 5 smaller dataframes, where each contains codes starting with the same 3 letters. Compare and contrast the codes in each group

**1. Add a column which contains the first 3 letters of the ICD-9 code**<br>
To get the first 3 letters of each row, write a function which takes an ICD-9 code and returns the first three characters. Then use the `diabetes.apply` method to run this function on every row.

In [ ]:
def ____(icd9):
    return ____

In [ ]:
diabetes["first_3"] = diabetes["code"].apply(____)

#### Bonus
How would would you do this with a lambda function?

In [ ]:
diabetes["first_3"] = diabetes["code"].apply(lambda __: ____)

#### 2. Get the counts of the first three characters

In [ ]:
diabetes.____.____

In [ ]:
diabetes.head()

#### 3. Subset the dataframe to only look at codes which start with the most common first 3 characters

In [ ]:
sub_df = diabetes[diabetes[____] == ____]
____.head()

# Patient diagnoses
Just like with labs or vitals, we can either analyze data from a **population** level or a **patient** level. Let's write some queries to do both.


### TODO
Let's look at the diagnoses for a specific patient. Write a query below to return the ICD-9 code and description for patient **286**.

In [ ]:
query = """
SELECT * 
FROM icd9
WHERE ____ = ____;
"""
df = pd.read_sql(query, conn)
df

## Most common diagnoses
Next, let's look not just at a single patient but the entire database. Let's write a query which will count how many times an ICD-9 code in MIMIC. We'll order the results by the count so that we can so the most common codes and compare them against the least common.

### TODO
Change the query below so that we group by code and description. Order by the count of codes in descending order.

In [ ]:
# What are the 10 most frequent diagnoses codes?
query = """
SELECT 
    code,
    description,
    ____ AS count
FROM icd9
____ __ ___, ____
ORDER BY ____ ____ ;
"""
code_counts = pd.read_sql(query, conn)

In [ ]:
# 10 most common codes
code_counts.head(10)

In [ ]:
# 10 least common codes
code_counts.tail(10)

### TODO
Let's plot a barplot of the 10 most common ICD-9 codes in the database.

- Create a new dataframe called `top_10_df` which contains the first 10 rows of `code_counts`
- Store the results of the query in a variable called `top_10_df`
- Call the `df.plot.bar` method to plot a barplot

In [ ]:
top_10_df = ____

In [ ]:
top_10_df.plot.bar(x=____, y=____)

## Patients with specific diagnoses
Now let's find the patients who have these 10 codes. Let's save this list of 10 codes as a Python variable and then we will add this into our query using the `.format()` method in Python.

The `format` method allows us to add a Python variable into our string. For example, in the example below, Python will print out a message using whatever values `first_name` and `last_name` have.

In [ ]:
first_name = "Alec"
last_name = "Chapman"
print("My name is {0} {1}.".format(first_name, last_name))

This makes it easier to construct strings using variables which might change or be too long to type out manually in a string.

### TODO
- Create a tuple of the 10 codes which we got from our query above (remember how to access a column of a pandas DataFrame?)
- Construct a query to get **unique** `subject_id`'s and ICD-9 codes
- Using `.format()`, add a where clause to limit to rows containing one of the 10 most common ICD-9 codes

In [ ]:
top_10_codes = tuple(____)
top_10_codes

In [ ]:
query = """
SELECT ____ 
    ____, 
    ____
FROM icd9
WHERE code IN ____
""".___(____)
print(query)

In [ ]:
# Now let's run the query
df = pd.read_sql(query, conn)
df.head()

In [ ]:
# Check that these are the same as top_10_codes
# If they are the same, this will give an empty set
set(df["code"]).difference(set(top_10_codes))

## Comorbidities
We're often interested in knowing about the **"comorbidity"** of a disease. A comorbidity is a condition which a patient has in addition to another condition. For example, if a patient has diabetes and they are also diagnosed with hypertension, then these two conditions would be comorbid. 

This is useful if we want to understand what conditions a population of patients might be at risk for based on the conditions they already have, or for measuring how certain diseases interact.

In this exercise we will calculate how frequently the 10 most common ICD-9 codes co-occur together.

### TODO
Find the most common conditions which patients who are diagnosed with hypertension also have. To do this, we'll have to use a **subquery** in order to first get the patients with hypertension, then get all the other codes which those patients have. With a complex query like this, it's helpful to write small pieces of the SQL first and then to put them together in a more complex statement. 

- Write a query to get the subject_id's of patients with **401.9, UNSPECIFIED ESSENTIAL HYPERTENSION**. Limit this to the top 10
- Write a separate query which gets the count of ICD-9 codes and descriptions
- Combine the two pieces into a single query using a **join**. Don't limit the number of rows to return. Give the query with subject_id's the alias `patients` and do a join with the rest of the table
- **Question**: What type of join will restrict the final results to only rows containing subject_ids found in the `patients` subquery?

#### 1. Write a query to get the unique id's of patients with hypertension

In [ ]:
htn_pats_query = """
SELECT ____
    ____
    FROM icd9
    WHERE ____ = ____
LIMIT 10
"""
htn_pats = pd.read_sql(htn_pats_query, conn)
htn_pats

#### 2. Write a query to get the counts of codes and descriptions (you can copy and paste from a query above)

In [ ]:
other_codes_query = """
SELECT 
    icd9.____
    icd9.____,
    ____ AS 'count'
FROM icd9
GROUP BY ____, ____
ORDER BY ____ desc
LIMIT 10
"""
other_codes = pd.read_sql(other_codes_query, conn)
other_codes

#### 3. Combine the two queries using a `join` statement
You can copy and paste parts of the two queries we just wrote. Consider where the subquery and where the main query should go.

In [ ]:
# Finally, combine the two queries 
query = """
SELECT 
    _____, 
    _____,
    COUNT(*) AS 'count'
FROM 
    icd9
    _____ _____
        /** Write the subquery here to get the subject_ids, call it `patients`*/
        (
        ____
        ) AS patients 
    ON icd9.subject_id = patients.subject_id
GROUP BY ____, ____
ORDER BY____ desc
LIMIT 10
"""

df = pd.read_sql(query, conn)
df

This is a useful query, and we may want to repeat it for other codes. Let's combine what we've done today and turn this query into a python function which takes the name of a code and a number and reurns the top comorbidities for that code.

#### TODO
Finish the function definition below. The function should take two arguments:
- `code`: The ICD-9 code to use in the subquery
- `count`: The number of comorbidities to return. Default 10

Copy and paste the query from above into the function. Then edit it and use the Python `.format()` method to add the user's `code` and `count` arguments to the query.

In [ ]:
def get_comorbidities(____, ____=10):
    query = """
        
        """.format(____, ____)
    return pd.read_sql(query, conn)

Now, let's call this function using a few different codes and look at what conditions these patients also have.

In [ ]:
# Type II Diabetes
get_comorbidities("250.00", 10)

In [ ]:
# Pneumonia
get_comorbidities("486", 20)

In [ ]:
# Asthma
get_comorbidities("493.90", 5)

In [ ]:
# Depression
"296.20"
get_comorbidities("296.20", 5)

In [ ]:
# Type II diabetes
"250.00"
get_comorbidities("250.00", 15)